In [7]:
import requests
import random
import json
import tqdm

url     = 'https://dev.api.chat.ask.eduworks.com/webhooks/rest/webhook'
DATA    = {
    "message"   : "How do you treat peach leaf curl?",
    "sender"    : str(random.randint(0, 9))
}

r = requests.post(url, json = DATA)

print(json.loads(r.text))

[{'recipient_id': '2', 'text': 'Here are my top results:'}, {'recipient_id': '2', 'custom': {'text': 'Here are my top results:', 'payload': 'collapsible', 'data': [{'title': '<p>1)<em>Peach Leaf Curl</em></br>(source: <a href="http://ipm.ucanr.edu/PMG/PESTNOTES/pn7426.html?src=exchbt" target="_blank">pestsIPM</a>)</p>', 'description': '<p><strong>Details</strong>: Peach leaf curl, also known as leaf curl, is a disease caused by the fungus Taphrina deformans. Peac</p></br><p><strong>Development</strong>: Peach leaf curl first appears in spring as reddish areas on developing leaves. These areas become th</p></br><p><strong>Management</strong>: To prevent peach leaf curl, use resistant peach and nectarine varieties where possible. (See the Res</p></br>'}, {'title': '<p>2)<em>Taphrina leaf curl</em></br>(source: <a href="http://ipm.ucanr.edu/PMG/GARDEN/PLANTS/DISEASES/taphlfcrl.html?src=exchbt" target="_blank">pestsDiseases</a>)</p>', 'description': '<p><strong>Details</strong>: Taphrina s

In [16]:
json.loads(r.text)

[{'recipient_id': '2', 'text': 'Here are my top results:'},
 {'recipient_id': '2',
  'custom': {'text': 'Here are my top results:',
   'payload': 'collapsible',
   'data': [{'title': '<p>1)<em>Peach Leaf Curl</em></br>(source: <a href="http://ipm.ucanr.edu/PMG/PESTNOTES/pn7426.html?src=exchbt" target="_blank">pestsIPM</a>)</p>',
     'description': '<p><strong>Details</strong>: Peach leaf curl, also known as leaf curl, is a disease caused by the fungus Taphrina deformans. Peac</p></br><p><strong>Development</strong>: Peach leaf curl first appears in spring as reddish areas on developing leaves. These areas become th</p></br><p><strong>Management</strong>: To prevent peach leaf curl, use resistant peach and nectarine varieties where possible. (See the Res</p></br>'},
    {'title': '<p>2)<em>Taphrina leaf curl</em></br>(source: <a href="http://ipm.ucanr.edu/PMG/GARDEN/PLANTS/DISEASES/taphlfcrl.html?src=exchbt" target="_blank">pestsDiseases</a>)</p>',
     'description': '<p><strong>Detai

In [8]:
import random
import re

sender = random.randint(0, 9)

URL     = 'https://dev.api.chat.ask.eduworks.com/webhooks/rest/webhook'
DATA    = {
    "message"   : "", 
    "sender"    : str(sender)
}
def run_tests(messages):
    results = []

    for m, _ in tqdm.tqdm(messages):
        DATA['message'] = m
        r = requests.post(URL, json = DATA)
        r = json.loads(r.text)
        
        assert isinstance(r, list)
        assert len(r) >= 3
        r = r[2]

        assert 'custom' in r
        r = r['custom']

        assert 'data' in r
        r = r['data']

        result = []
        for e in r:
            title   = re.findall("<em>(.*?)</em>"           , e['title'])[0]
            link    = re.findall("href=[\"\'](.*?)[\"\']"   , e['title'])[0]
            result.append((title, link))
        
        results.append(result)

        DATA['message'] = '/intent_affirm'
        r = requests.post(URL, json = DATA)
        r = json.loads(r.text)

        assert len(r) == 1
        r = r[0]

        assert 'text' in r
        r = r['text']

        assert 'Anything else I can help with?' == r

    return results


In [9]:
import pandas as pd

df = pd.read_csv('./data/UCIPM_Questions_2020_08_19.csv')

df['request'] = df['Question'   ].apply(lambda x: x.split('\n')[0]              )
df['answer' ] = df['URL'        ].apply(lambda x: x.split('\n')                 )
df['answer' ] = df['answer'     ].apply(lambda x: [x1.split('?')[0] for x1 in x])

df = df[['request', 'answer']]
df.sample(10)

,request,answer
44,How do I deter deer from my garden?,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn74117.html]
8,There are roof rats getting into my home. What...,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn74106.html]
34,If my palm tree has little black spots on it d...,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn74148.html]
12,What does rust on roses look like?,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7463.html]
36,How do you set a gopher trap?,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7433.htm...
30,Can I plant vegetables in the shade?,[http://ipm.ucanr.edu/PMG/GARDEN/VEGES/CULTURA...
14,There are tons of little green bugs on my rose...,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7404.htm...
22,I planted seeds but they never came up. What h...,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn74132.html]
38,What do scale insects look like?,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7408.html]
18,What is biting me in bed?,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7454.htm...


In [10]:
messages    = df.values.tolist()

results     = run_tests(messages)

  0%|          | 0/50 [00:01<?, ?it/s]


AssertionError: 

In [33]:
scores = []
for i, r in enumerate(results):
    answers = messages[i][1]
    topn = [False, False, False]
    for i1, r1 in enumerate(r):
        if r1[1].split('?')[0] in answers:
            if i1 == 0:
                topn[0] = True
            if i1 < 3:
                topn[1] = True
            topn[2] = True
    scores.append(topn)

In [34]:
top1 = 0
top3 = 0
top5 = 0
for topn in scores:
    if topn[0]: top1 += 1
    if topn[1]: top3 += 1
    if topn[2]: top5 += 1

print(f'Out of {len(results)} results, following correct:'  )
print(f'Top 1: {top1:<3d} ({top1/len(results) * 100:<.2f}%)')
print(f'Top 3: {top3:<3d} ({top3/len(results) * 100:<.2f}%)')
print(f'Top 5: {top5:<3d} ({top5/len(results) * 100:<.2f}%)')

Out of 50 results, following correct:
Top 1: 24  (48.00%)
Top 3: 28  (56.00%)
Top 5: 32  (64.00%)
